## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [2]:
NAME = "Κωνσταντίνος Ανδρέου"
AEM = "9521"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [3]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [4]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

('test_set_noclass.csv', <http.client.HTTPMessage at 0x1f985a4f4f0>)

In [6]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure weighted and balanced accuracy of your models. You can use [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) and select both metrics to be measured during the evaluation. Otherwise, you can use [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold).

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three **simple** estimators/classifiers. Test both soft and hard voting and choose the best one. Consider as simple estimators the following:


*   Decision Trees
*   Linear Models
*   Probabilistic Models (Naive Bayes)
*   KNN Models  

In [5]:
# BEGIN CODE HERE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score,cross_validate

cls1 = DecisionTreeClassifier(random_state=42,max_depth=7) # Classifier #1 
cls2 = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)# Classifier #2 
cls3 = LogisticRegression(random_state=42,n_jobs=-1) # Classifier #1
soft_vcls = VotingClassifier(estimators=[("dtree",cls1),("knn",cls2),("lr",cls3)],n_jobs=-1,voting="soft") # Voting Classifier
hard_vcls = VotingClassifier(estimators=[("dtree",cls1),("knn",cls2),("lr",cls3)],n_jobs=-1,voting="hard") # Voting Classifier

svlcs_scores = cross_validate(soft_vcls, X, y, cv=10, scoring=('f1_weighted','balanced_accuracy'), n_jobs=-1)
s_avg_fmeasure = sum(cross_val_score(soft_vcls,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2))/10 # The average f-measure
s_avg_accuracy = sum(cross_val_score(soft_vcls,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2))/10 # The average accuracy
# s_avg_fmeasure = cv_results['test_f1_weighted'].mean()  
# s_avg_accuracy = cv_results['test_balanced_accuracy'].mean() 

hvlcs_scores = cross_validate(hard_vcls, X, y, cv=10, scoring=('f1_weighted','balanced_accuracy'), n_jobs=-1)
h_avg_fmeasure = sum(cross_val_score(hard_vcls,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2))/10 # The average f-measure
h_avg_accuracy = sum(cross_val_score(hard_vcls,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2))/10 # The average 
# h_avg_fmeasure = cv_results['test_f1_weighted'].mean()  
# h_avg_accuracy = cv_results['test_balanced_accuracy'].mean() 
#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   35.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.8s finished


In [6]:
print("Classifier:")
print(soft_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(s_avg_fmeasure,4), round(s_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('dtree',
                              DecisionTreeClassifier(max_depth=7,
                                                     random_state=42)),
                             ('knn', KNeighborsClassifier(n_jobs=-1)),
                             ('lr',
                              LogisticRegression(n_jobs=-1, random_state=42))],
                 n_jobs=-1, voting='soft')
F1 Weighted-Score: 0.8312 & Balanced Accuracy: 0.8235


You should achive above 82% (Soft Voting Classifier)

In [7]:
print("Classifier:")
print(hard_vcls)
print("F1 Weighted-Score: {} & Balanced Accuracy: {}".format(round(h_avg_fmeasure,4), round(h_avg_accuracy,4)))

Classifier:
VotingClassifier(estimators=[('dtree',
                              DecisionTreeClassifier(max_depth=7,
                                                     random_state=42)),
                             ('knn', KNeighborsClassifier(n_jobs=-1)),
                             ('lr',
                              LogisticRegression(n_jobs=-1, random_state=42))],
                 n_jobs=-1)
F1 Weighted-Score: 0.8245 & Balanced Accuracy: 0.816


You should achieve above 80% in both! (Hard Voting Classifier)

### 1.2 Stacking ###
Create a stacking classifier which uses two more complex estimators. Try different simple classifiers (like the ones mentioned before) for the combination of the initial estimators. Report your results in the following cell.

Consider as complex estimators the following:

*   Random Forest
*   SVM
*   Gradient Boosting
*   MLP




In [8]:
# BEGIN CODE HERE
from sklearn.ensemble import StackingClassifier,GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Classifier #1 "
cls1 = RandomForestClassifier(random_state= 42,n_jobs=-1)
# Classifier #2 "
cls2 = GradientBoostingClassifier(random_state=42)
cls3 = SVC(random_state=42) # Classifier #3 (Optional)
scls = StackingClassifier(estimators=[('RFC',cls1),('GBC',cls2),('SVC',cls3)],n_jobs=-1) # Stacking Classifier
# scls = VotingClassifier(estimators=[("dtree",cls1),("knn",cls2)],n_jobs=-1,voting="soft")
scores =  cross_validate(scls, X, y, cv=10, scoring=('f1_weighted','balanced_accuracy'), n_jobs=-1)
avg_fmeasure = sum(cross_val_score(scls,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2))/10 # The average f-measure
avg_accuracy = sum(cross_val_score(scls,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2))/10 # The average accuracy
# avg_fmeasure = cv_results['test_f1_weighted'].mean()  
# avg_accuracy = cv_results['test_balanced_accuracy'].mean() 
#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 53.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 52.3min finished


In [9]:
print("Classifier:")
print(scls)
print("F1 Weighted Score: {} & Balanced Accuracy: {}".format(round(avg_fmeasure,4), round(avg_accuracy,4)))

Classifier:
StackingClassifier(estimators=[('RFC',
                                RandomForestClassifier(n_jobs=-1,
                                                       random_state=42)),
                               ('GBC',
                                GradientBoostingClassifier(random_state=42)),
                               ('SVC', SVC(random_state=42))],
                   n_jobs=-1)
F1 Weighted Score: 0.8627 & Balanced Accuracy: 0.857


You should achieve above 85% in both

Όσον αφορά το voting  βρίκα καλύτερα αποτελέσματα στο soft voting.

Soft Voting F1 Weighted-Score: 0.8312 & Balanced Accuracy: 0.8235
Hard Voting F1 Weighted-Score: 0.8245 & Balanced Accuracy: 0.816

Χρησιμοποίησα decision tree , knn and logistic regression με ελάχιστες παραμέτρου (να μην έχω verfit) και έβγαλα επιθυμητά αποτελέσματα.

Στο stacking μετά από εξονυχιστικά grid search και άπειρες δοκιμές βρίκα επιθυμητά αποτελέσματα με random forest , gradient boost , SVC and Stacking Classifier χωρίς παραμέτρους. Για το stacking άφησα τον meta estimator default ο οποίος είναι logistic regression.

Stacking F1 Weighted Score: 0.8627 & Balanced Accuracy: 0.857

Σκοπός μου ήταν να χρησιμοποιήσω estimators με διαφορετικές αδυναμίες και πρωτερίμετα με σκοπός σε καθένα από τα subsets να έχω τουλάχιστον κάποια σωστά αποτελέσματα.


## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1_weighted/balanced_accuracy score. The dictionaries should contain four different elements.  

In [10]:
# BEGIN CODE HERE
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier

ens1 = RandomForestClassifier(max_depth=7,n_jobs=-1,random_state=42)
ens2 = AdaBoostClassifier(random_state=42)
ens3 = GradientBoostingClassifier(max_depth=3,random_state=42)
tree = DecisionTreeClassifier(max_depth=7,random_state=42)

f_measures = dict()
f_score=cross_val_score(tree,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2)
f_measures["Simple Deciscion"]=f_score.mean()
f_score=cross_val_score(ens1,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2)
f_measures["Ensemble with Random Forest"]=f_score.mean()
f_score=cross_val_score(ens2,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2)
f_measures["Ensemble with Adaboost"]=f_score.mean()
f_score=cross_val_score(ens3,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2)
f_measures["Ensemble with Gradientboost"]=f_score.mean()

accuracies = dict()
# Example f_measures = {'Simple Decision': 0.8551, 'Ensemble with random ...': 0.92, ...}
acc=cross_val_score(tree,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2)
accuracies["Simple Deciscion"]=acc.mean()
acc=cross_val_score(ens1,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2)
accuracies["Ensemble with Random Forest"]=acc.mean()
acc=cross_val_score(ens2,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2)
accuracies["Ensemble with Adaboost"]=acc.mean()
acc=cross_val_score(ens3,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2)
accuracies["Ensemble with Gradientboost"]=acc.mean()
#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  7.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10

In [12]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1 Weighted:{}".format(name,round(score,4)))
for name,score in accuracies.items():
    print("Classifier:{} -  BalancedAccuracy:{}".format(name,round(score,4)))

RandomForestClassifier(max_depth=7, n_jobs=-1, random_state=42)
AdaBoostClassifier(random_state=42)
GradientBoostingClassifier(random_state=42)
DecisionTreeClassifier(max_depth=7, random_state=42)
Classifier:Simple Deciscion -  F1 Weighted:0.7165
Classifier:Ensemble with Random Forest -  F1 Weighted:0.7582
Classifier:Ensemble with Adaboost -  F1 Weighted:0.7825
Classifier:Ensemble with Gradientboost -  F1 Weighted:0.8175
Classifier:Simple Deciscion -  BalancedAccuracy:0.704
Classifier:Ensemble with Random Forest -  BalancedAccuracy:0.7371
Classifier:Ensemble with Adaboost -  BalancedAccuracy:0.7739
Classifier:Ensemble with Gradientboost -  BalancedAccuracy:0.8064


**2.2** Describe your classifiers and your results.

YOUR ANSWER HERE
Deciscion Trees:Τα δέντρα απόφασης χρησιμοποιούν πολλαπλούς αλγόριθμους για να αποφασίσουν να χωρίσουν ανάλογα με το είδος του δέντρου , χρησιμοποιούν εντροπία ή gini και information gain για να διαχωρίσουν έναν κόμβο σε δύο ή περισσότερους υπο-κόμβους. Υπολογίζεται για το κάθε χαρακτηριστικό ξεχωριστά και έτσι αποφασίζει ποιός θα είναι ο πατέρας και ποιό το παιδί. Αυτή η διαδικασία συνεχίζεται μέχρι να κατασκευαστεί το δέντρο.
 
Random Forest: Ο random forest δημιουργεί πολλά δέντρα αποφάσεων για να πάρει μια πιο ακριβή και σταθερή πρόβλεψη.Επιλέγει τυχαία χαρακτηριστικά για το κάθε δέντρο που χρησιμοποιεί από bootstrap δεδομένα. O random forest έχει σχεδόν τις ίδιες υπερπαραμέτρους με ένα δέντρο αποφάσεων ή έναν bagging classifier.Το τυχαίο δάσος προσθέτει επιπλέον τυχαιότητα στο μοντέλο, ενώ καλλιεργεί τα δέντρα. Στο τέλος η απόφαση γίνεται με βάση τη προβλέψανε τα δέντρα του δάσους.Στο τέλος χρησιμοποιά τα out of the bag στιγμιότυπα για να αξιολογήσει το μοντέλο.
 
Gradient Boost: Το Gradient Boosting είναι ένας επαναληπτικός functional gradient algorithm, δηλαδή ένας αλγόριθμος που ελαχιστοποιεί μια συνάρτηση απώλειας επιλέγοντας επαναληπτικά μια συνάρτηση που δείχνει προς την negative gradient.Παράγει δέντρα κάθε φορά προσπαθώντας να διορθώσει τα λαθοι του προηγούμενου δέντρου μειώνοντας το σφάλμα του στις προβλέψεις και δίνοντας βαροι στα δέντρα ανάλογα με τα πιό σωστά.
 
Adaboost:Ο ταξινομητής αυτός , δημιουργεί stumps (δέντρα με 1 γονιό , 2 παιδιά) για κάθε χαρακτηριστικό και σύμφωνα με τα λάθοι που έκανε , δίνει βάρος στα δεδομένα που προέβλεψε λάθος έτσι ώστε στην επόμενη δημιουργία να προσπαθήσει να τα διορθώσει. Είναι επίσης ένας επαναληπτικός αλγόριθμος ο οποίος προσπαθεί να διορθώσει τα λάθη του.
 
 
Εφάρμοσα τους ποιό πάνω ταξινομητές που χρησιμοποιούν δέντρα για την δημιουργία τους. Οι μόνοι περιορισμοί που χρησιμοποίησα είναι τα max depth για να αποφύγω overfitting . Δεν χρειάστηκαν άλλοι περιορισμοί καθώς βγάζω ικανοποιητικά αποτελέσματα. Ο καλύτερος στην περίπτωση μου είναι ο gradient boost. Είναι λογικό γιατί παρά την αυξημένη πολυπλοκότητα του , προσπαθεί να διορθώσει τα λάθη του και διορθώνει κάποιες από τις αδυναμίες των άλλων μοντέλων.
 
Γενικά προσπάθησα να χρησιμοποιήσω ταξινομητές με διαφορετικές συμπεριφορές για να διορθώνει ο ένας τις αδυναμίες του άλλου και βλέπω ότι τα καλύτερα αποτελέσματα τα έχω με τον gradient boost


**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

YOUR ANSWER HERE
Στο bagging εκπαιδεύονται μοντέλα ανεξάρτητα μεταξύ τους άρα η διαδικασία μπορεί να γίνει και παράλληλα(παραλληλισμός), έτσι μειώνεται αρκετά ο χρόνος εκπαίδευσης. Δυστυχώς αυτό στα άλλα boosting μοντέλα δεν μπορούμε να το κάνουμε γιατί κάθε δέντρο που δημιουργείται είναι σε σχέση με το προηγούμενο. n_jobs = -1

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure (weighted) & balanced accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve a balanced accuracy over 83-84%?

In [12]:
# BEGIN CODE HERE
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score,cross_validate

# cls1 = RandomForestClassifier(random_state= 42,n_jobs=-1)
# Classifier #2 "
# cls2 = GradientBoostingClassifier(random_state=42)
# cls3 = SVC(random_state=42) # Classifier #3 (Optional)
# scls = StackingClassifier(estimators=[('RFC',cls1),('GBC',cls2),('SVC',cls3)],n_jobs=-1) # Stacking Classifier
# # scls = VotingClassifier(estimators=[("dtree",cls1),("knn",cls2)],n_jobs=-1,voting="soft")
# scores =  cross_validate(scls, X, y, cv=10, scoring=('f1_weighted','balanced_accuracy'), n_jobs=-1)


mlp1 = MLPClassifier(random_state=42, activation='relu', hidden_layer_sizes=(50, 50),
                     learning_rate='constant')
mlp2 = MLPClassifier(random_state=42, activation='relu', hidden_layer_sizes=(50, 50, 50),
                     learning_rate='constant')
mlp3 = MLPClassifier(random_state=42, activation='relu', hidden_layer_sizes=(50, 50, 50, 50),
                     learning_rate='constant')
# Χρησιμοποιήθηκε voting και όχι stacking λόγω χρόνου...
cls = VotingClassifier(estimators=[('mlp1', mlp1), ('mlp2', mlp2), ('mlp3', mlp3) ],
                            voting='hard')
best_cls = Pipeline([
    ('scale', StandardScaler()),
    ('voting', cls)])

# Cross validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_results = cross_validate(best_cls, X, y, cv=10, n_jobs=-1, verbose=1, scoring=['balanced_accuracy', 'f1_weighted'])

print(cv_results)

best_fmeasure =sum(cross_val_score(best_cls,X,y,cv=10,scoring="f1_weighted",n_jobs=-1,verbose=2))/10
best_accuracy = sum(cross_val_score(best_cls,X,y,cv=10,scoring="balanced_accuracy",n_jobs=-1,verbose=2))/10

# best_fmeasure = cv_results['test_f1_weighted'].mean()  
# best_accuracy = cv_results['test_balanced_accuracy'].mean() 

#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.2min finished


{'fit_time': array([114.92062807, 102.15733719, 117.88398814, 120.37117958,
        99.61327195, 121.2740984 , 120.09749103, 116.92477536,
        88.10658216,  71.27705884]), 'score_time': array([0.17504573, 0.17805672, 0.18105221, 0.17394137, 0.17388153,
       0.17703176, 0.15895104, 0.17488432, 0.14762664, 0.07384515]), 'test_balanced_accuracy': array([0.82626196, 0.84448059, 0.86644357, 0.85401699, 0.85777305,
       0.86210245, 0.874694  , 0.84107571, 0.8389492 , 0.84974623]), 'test_f1_weighted': array([0.83302802, 0.85378895, 0.87131403, 0.86137086, 0.86397067,
       0.8645585 , 0.87833458, 0.84898478, 0.84063263, 0.85640104])}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.1min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.5min finished


In [13]:
print("Classifier:")
#print(best_cls)
print("F1 Weighted-Score:{} & Balanced Accuracy:{}".format(best_fmeasure, best_accuracy))

Classifier:
F1 Weighted-Score:0.8572384066630363 & Balanced Accuracy:0.8515543746345948


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

Αρχικά προσπάθησα να ξανακάνω gridsearch για τα καλύτερα hyper parameters των πιο καλών μοντέλων μου βρήκα πάνω , δηλαδή stacking με andom forest , gradientboost , SVC and StackingClassifier αλλά δεν ήμουν ικανοποιημένος με τα αποτελέσματά μου κυρίως γιατί ήθελε πολλή ώρα να τρέξει. Έτσι πειραματίστηκα και με άλλους αλγόριθμους όπως τα νευρωνικά και με την βοήθεια του internet δοκίμασα να κάνω stacking με νευρωνικά διαφορετικών hidden layers . https://stackoverflow.com/questions/34723489/how-do-multiple-hidden-layers-in-a-neural-network-improve-its-ability-to-learn . Γιαυτό για να μην βάλω απλά πολλά layers και να έχω verfiting έκανα stacking 3 διαφορετικών για να έχουν γνώμη όλοι στο τελικό αποτέλεσμα. Τελικά έχω καλύτερα αποτελέσματα με αυτό τον τρόπο.Piplene και scaler δεν χρειαζόταν αλλά επειδή έχω σκοπό να το χρησιμοποιήσω και στην 3.3 το έβαλα ος παράδειγμά. Παρόλο που δεν χρειάζεται στην συγκεκριμένη εργασία να κάνουμε preprocessing όταν πρέπει να κάνουμε deploy το μοντέλο μας για νομιμοποιηθεί , θα πρέπει να επεξεργαστούμε τα δεδομένα.

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [ ]:
# BEGIN CODE HERE
cls = best_cls

cls.fit(X, y)
# predictions_list = list(cls.predict(test_set))
#END CODE HERE
test_set = pd.read_csv("test_set_noclass.csv")
predictions = cls.predict(test_set)

AttributeError: 'ellipsis' object has no attribute 'predict'

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

Μπορούσα να χρησημοποιήσω τους πύνακες των scores για να βρώ mean() αλλά άργισα να το δώ και τα έτρεξα με έξτρα εντολές, είναι λάθος αλλά δεν προλαμβένω να τρέξω ξανά όλο το πρόγραμμα. Θα γλύτωνα και αρκετό χρώνο.

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
if False:
  from sklearn.metrics import f1_score, balanced_accuracy_score
  final_test_set = pd.read_csv('test_set.csv')
  ground_truth = final_test_set['CLASS']
  print("Balanced Accuracy: {}".format(balanced_accuracy_score(predictions, ground_truth)))
  print("F1 Weighted-Score: {}".format(f1_score(predictions, ground_truth, average='weighted')))

Both should aim above 85%!